# Cross Correlation

The general form of the cross correlation with integration:
\begin{equation}
{\displaystyle (f\star g)(\tau )\ = \int _{-\infty }^{\infty }{{f^{\star}(t)}}g(t+\tau )\,dt}
\end{equation}
This can be written in discrete form as:
\begin{equation}
{\displaystyle (f\star g)[n]\ = \sum _{m=-\infty }^{\infty }{{f^{\star}[m]}}g[m+n]}
\end{equation}

In [ ]:
from matplotlib import pylab
from matplotlib import pyplot as plt
pylab.rcParams['savefig.dpi'] = 300
import numpy as np

In [ ]:
from gps_helper.prn import PRN
from sk_dsp_comm import sigsys as ss
from sk_dsp_comm import digitalcom as dc
from caf_verilog.quantizer import quantize

## Test Signals

In [ ]:
prn = PRN(10)
prn2 = PRN(20)
fs = 625e3
Ns = fs / 125e3
prn_seq = prn.prn_seq()
prn_seq2 = prn2.prn_seq()
prn_seq,b = ss.nrz_bits2(np.array(prn_seq), Ns)
prn_seq2,b2 = ss.nrz_bits2(np.array(prn_seq2), Ns)

In [ ]:
Px,f = plt.psd(prn_seq, 2**12, Fs=fs)
plt.plot(f, 10*np.log10(Px))

## Autocorrelation

In [ ]:
r, lags = dc.xcorr(prn_seq, prn_seq, 100)
plt.plot(lags, abs(r)) # r -> abs

## Time Shifted Signals

In [ ]:
r, lags = dc.xcorr(np.roll(prn_seq, 50), prn_seq, 100)
plt.plot(lags, abs(r))

In [ ]:
r, lags = dc.xcorr(np.roll(prn_seq, -50), prn_seq, 100)
plt.plot(lags, abs(r))

## No Correlation

In [ ]:
r_nc, lags_nc = dc.xcorr(prn_seq, prn_seq2, 100)
plt.plot(lags_nc, abs(r_nc))
plt.ylim([0, 1])

## Calculation Space Visualization

In [ ]:
from caf_verilog.xcorr import size_visualization
size_visualization(prn_seq[:10], prn_seq[:10], 5)

## Simple Cross Correlation

In [ ]:
from caf_verilog.xcorr import simple_xcorr
r, lags = simple_xcorr(prn_seq, prn_seq, 100)
plt.plot(lags, r)

### Time Shifted Signals

In [ ]:
r, lags = simple_xcorr(prn_seq, np.roll(prn_seq, 50), 100)
plt.plot(lags, abs(np.array(r)))

In [ ]:
r, lags = simple_xcorr(prn_seq, np.roll(prn_seq, -50), 100)
plt.plot(lags, abs(np.array(r)))

### No Correlation

In [ ]:
r, lags = simple_xcorr(prn_seq, prn_seq2, 100)
plt.plot(lags, abs(np.array(r)))
plt.ylim([0, 5000])

## Dot Product Method

To ensure the integration time is filled, the secondary or received signal must be twice the length of the reference signal.

In [ ]:
from caf_verilog.sim_helper import sim_shift
center = 300
corr_length = 250
shift = 25
ref, rec = sim_shift(prn_seq, center, corr_length, shift=shift, padding=True)

In [ ]:
f, axarr = plt.subplots(2, sharex=True, gridspec_kw={'hspace': 0})
axarr[0].plot(ref)
axarr[1].plot(rec)
plt.xlabel("Sample Number")
plt.savefig('prn_seq.png')

In [ ]:
from caf_verilog.xcorr import dot_xcorr
ref, rec = sim_shift(prn_seq, center, corr_length, shift=shift)
rr = dot_xcorr(ref, rec)
rr = np.array(rr)

In [ ]:
rxy, lags = dc.xcorr(ref, rec, 1000)
plt.plot(abs(rxy))
plt.xlabel("Center Offset (Samples)")
plt.grid();
plt.savefig('xcorr.png')

In [ ]:
fig, axs = plt.subplots(2, sharey=True)
axs[0].plot(abs(rr))
axs[0].grid(True)
axs[1].plot(abs(rr))
axs[1].set_xlim([80, 120])
axs[1].set_xlabel('Inverse Center Offset (Samples)')
axs[1].grid(True)
fig.savefig('xcorr_250.png')

In [ ]:
np.argmax(rr)

In [ ]:
from caf_verilog.xcorr import XCorr
xc = XCorr(ref, rec, output_dir='.')

In [ ]:
xc.gen_tb()